### import SQL
--------------
사전작업
- 1) Atlas Cohort 생성 (Target / Case)
- 2) DB에서 Target / Case 병합 Table 생성
-------------
변경사항
- 1) current_date
- 2) outcome_name ; 변경하면서 다시 재실행 
- 3) db 정보 및 dbTableSchema 변경
--------------

In [1]:
# @변경 필요한 변수

current_date = 20220317
# outcome_list=['meloxicam', 'celecoxib', 'valproic_acid, lamotrigine']
outcome_name = 'meloxicam' 
domains = ['measurement', 'drug', 'procedure', 'condition']

In [2]:
import os
import pathlib
import sys
sys.path.append("..")

current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
output_dir = pathlib.Path('{}/data/{}/importsql/{}/'.format(parent_dir, current_date, outcome_name))
pathlib.Path.mkdir(output_dir, mode=0o777, parents=True, exist_ok=True)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import os
import pandas as pd
import numpy as np
#!pip install pymssql
import pymssql
from datetime import timedelta

pd.set_option('display.max_colwidth', -1)  #각 컬럼 width 최대로 
pd.set_option('display.max_rows', 50)      # display 50개 까지 

C:\Users\suncheolheo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if sys.path[0] == '':


### Postgresql (계정유출 주의)

In [4]:
# import pandas.io.sql as psql
# import psycogp2 as pg
# conn = pg.connect(host='111.1.11.11', dbname='cdm', user='1111', password='1111', port='5432')
# print('connection')

# sql = 'select * from cdm.cdm.person'
# population = psql.read(sql=sql1, con=conn)
# population.to_csv('population.txt', index=False)

### Mssql (계정유출 주의)

In [5]:
import pymssql
#conn= pymssql.connect(server="111.111.111.111", user="user", password="password", port=22, database='db', as_dict=False)
conn= pymssql.connect(server="61.252.54.76", user="Atlas", password="dhlab2021!", port=14333, database='CDM', as_dict=False)
print("connection")

connection


### Table Load from DB

In [6]:
tnPopulation = 'temp.dbo.person_{}'.format(outcome_name)
tnMeasurement = 'CDM.dbo.measurement'
tnDrug = 'CDM.dbo.drug_exposure'
tnProcedure = 'CDM.dbo.procedure_occurrence'
tnCondition = 'CDM.dbo.condition_occurrence'
tnConcept = 'CDM.dbo.concept'

In [7]:
# 간독성 Patient 불러오기
sql_query = 'select * from {}'.format(tnPopulation)
population_df = pd.read_sql(sql=sql_query, con=conn)

# 결과확인 
print('success : {}'.format(len(population_df)))

success : 6978


### Table Load from DB (Measurement / drug / procedure / concept)

In [8]:
# Measurement / drug / procedure / concept 불러오기 
sql_query ="select person_id, measurement_concept_id, measurement_date, value_as_number from {} ".format(tnMeasurement) +\
    "where measurement_concept_id!=0 and value_as_number is not null and person_id in (select distinct person_id from {})".format(tnPopulation)
meas_df = pd.read_sql(sql=sql_query, con=conn)

sql_query="select person_id, drug_concept_id, drug_exposure_start_date, quantity from {} ".format(tnDrug) +\
    "where drug_concept_id!=0 and quantity is not null and person_id in (select distinct person_id from {})".format(tnPopulation)
drug_df = pd.read_sql(sql=sql_query, con=conn)

sql_query="select person_id, procedure_concept_id, procedure_date from {} ".format(tnProcedure) +\
    "where procedure_concept_id!=0 and person_id in (select distinct person_id from {} )".format(tnPopulation)
proc_df = pd.read_sql(sql=sql_query, con=conn)

sql_query="select person_id, condition_concept_id, condition_start_date from {} ".format(tnCondition) +\
    "where condition_concept_id!=0 and person_id in (select distinct person_id from {} )".format(tnPopulation)
cond_df = pd.read_sql(sql=sql_query, con=conn)

# 결과확인 
print('success : {}'.format(len(meas_df), len(drug_df), len(proc_df), len(cond_df)))

success : 29157876


### Table Load from DB (Concept)

In [9]:
## concept table 전체 불러오기 
# sql_query="select distinct concept_id, concept_name from {}".format(tnConcept)
# concept_df = pd.read_sql(sql=sql_query, con=conn)

## concept table 일부 불러오기 
sql_query="select distinct concept_id, concept_name from {} ".format(tnConcept) +\
    "where concept_id !=0 and concept_id in (select distinct measurement_concept_id from {})".format(tnMeasurement)
concept_meas_df = pd.read_sql(sql=sql_query, con=conn)
sql_query="select distinct concept_id, concept_name from {} ".format(tnConcept) +\
    "where concept_id !=0 and concept_id in (select distinct drug_concept_id from {})".format(tnDrug)
concept_drug_df = pd.read_sql(sql=sql_query, con=conn)
sql_query="select distinct concept_id, concept_name from {} ".format(tnConcept) +\
    "where concept_id !=0 and concept_id in (select distinct procedure_concept_id from {})".format(tnProcedure)
concept_proc_df = pd.read_sql(sql=sql_query, con=conn)
sql_query="select distinct concept_id, concept_name from {} ".format(tnConcept) +\
    "where concept_id !=0 and concept_id in (select distinct condition_concept_id from {})".format(tnCondition)
concept_cond_df = pd.read_sql(sql=sql_query, con=conn)

print(len(concept_meas_df), len(concept_drug_df), len(concept_proc_df), len(concept_cond_df))
concept_df = pd.concat([concept_meas_df, concept_drug_df, concept_proc_df, concept_cond_df], axis=0)
concept_df

conn.close()

579 1533 1265 2636


### Save dataset

In [10]:
population_df.to_csv('{}/population.txt'.format(output_dir),index=False)
meas_df.to_csv('{}/measurement.txt'.format(output_dir),index=False)
drug_df.to_csv('{}/drug.txt'.format(output_dir),index=False)
proc_df.to_csv('{}/procedure.txt'.format(output_dir),index=False)
cond_df.to_csv('{}/condition.txt'.format(output_dir),index=False)
concept_df.to_csv('{}/concept.txt'.format(output_dir),index=False)

### Load dataset

In [11]:
# 데이터 불러오기
population_df=pd.read_csv('{}/population.txt'.format(output_dir))
meas_df=pd.read_csv('{}/measurement.txt'.format(output_dir))
drug_df=pd.read_csv('{}/drug.txt'.format(output_dir))
proc_df=pd.read_csv('{}/procedure.txt'.format(output_dir))
cond_df=pd.read_csv('{}/condition.txt'.format(output_dir))
concept_df=pd.read_csv('{}/concept.txt'.format(output_dir))

In [12]:
population_df['label'] = (~population_df['first_abnormal_date'].isnull()).astype(int)
population_df.rename(columns={'gender_source_value':'sex'}, inplace=True)
population_df['sex'].replace(['F', 'Female'], 0, inplace=True)
population_df['sex'].replace(['M', 'Male'], 1, inplace=True)

In [13]:
### Get only used columns
meas_df = meas_df[["person_id","measurement_concept_id","measurement_date","value_as_number"]]
drug_df = drug_df[["person_id","drug_concept_id","drug_exposure_start_date","quantity"]]
proc_df = proc_df[["person_id","procedure_concept_id","procedure_date"]]
cond_df = cond_df[["person_id","condition_concept_id","condition_start_date"]]
concept_df = concept_df[["concept_id","concept_name"]]

meas_df.drop_duplicates(inplace=True)
drug_df.drop_duplicates(inplace=True)
proc_df.drop_duplicates(inplace=True)
cond_df.drop_duplicates(inplace=True)

### @use common terminology.
meas_df.rename(columns={'measurement_concept_id':'concept_id','measurement_date':'concept_date','value_as_number':'concept_value'}, inplace=True)
drug_df.rename(columns={'drug_concept_id':'concept_id','drug_exposure_start_date':'concept_date','quantity':'concept_value'}, inplace=True)
proc_df.rename(columns={'procedure_concept_id':'concept_id','procedure_date':'concept_date'}, inplace=True)
cond_df.rename(columns={'condition_concept_id':'concept_id','condition_start_date':'concept_date'}, inplace=True)

### population + domain
meas_df = pd.merge(population_df, meas_df, left_on=["person_id"], right_on=["person_id"], how="inner").reset_index(drop=True)
drug_df = pd.merge(population_df, drug_df, left_on=["person_id"], right_on=["person_id"], how="inner").reset_index(drop=True)
proc_df = pd.merge(population_df, proc_df, left_on=["person_id"], right_on=["person_id"], how="inner").reset_index(drop=True)
cond_df = pd.merge(population_df, cond_df, left_on=["person_id"], right_on=["person_id"], how="inner").reset_index(drop=True)


In [14]:
### Get only used dates
def cohortConditionSetting(domain_df, pre_observation_period, post_observation_peroid):
    from datetime import timedelta
    prev_len = len(domain_df)
    domain_df['cohort_start_date'] = pd.to_datetime(domain_df['cohort_start_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')
    domain_df['first_abnormal_date'] = pd.to_datetime(domain_df['first_abnormal_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')
    domain_df['concept_date'] = pd.to_datetime(domain_df['concept_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')
    # condition 1) Select patients with first adverse events within 2 months of cohort initiation.
    domain_df = domain_df[(domain_df['cohort_start_date']<=domain_df['concept_date']+timedelta(days=pre_observation_period))]
    # condition 2) Delete data before the cohort start date.
    domain_df = domain_df[(domain_df['concept_date']<=domain_df['cohort_start_date']+timedelta(days=post_observation_peroid))]
    # condition 3) Delete data after first_abnormal_date (Except when there is no first abnormal date.)
    domain_df = domain_df[~(domain_df['first_abnormal_date']<domain_df['concept_date'])]
    domain_df = domain_df[~(domain_df['first_abnormal_date']-domain_df['cohort_start_date']>timedelta(days=post_observation_peroid))]
    domain_df = domain_df.reset_index(drop=True)
    curr_len = len(domain_df)
    print('{} > {}'.format(prev_len, curr_len))
    return domain_df    

meas_df = cohortConditionSetting(meas_df, pre_observation_period=60, post_observation_peroid=60)
drug_df = cohortConditionSetting(drug_df, pre_observation_period=60, post_observation_peroid=60)
proc_df = cohortConditionSetting(proc_df, pre_observation_period=60, post_observation_peroid=60)
cond_df = cohortConditionSetting(cond_df, pre_observation_period=60, post_observation_peroid=60)


11441336 > 3029160
3382776 > 972356
512955 > 127812
1608848 > 241080


In [15]:

### population + domain + concept
meas_df = pd.merge(meas_df,concept_df,left_on=["concept_id"],right_on=["concept_id"],how="inner")
drug_df = pd.merge(drug_df,concept_df,left_on=["concept_id"],right_on=["concept_id"],how="inner")
proc_df = pd.merge(proc_df,concept_df,left_on=["concept_id"],right_on=["concept_id"],how="inner")
cond_df = pd.merge(cond_df,concept_df,left_on=["concept_id"],right_on=["concept_id"],how="inner")

meas_df['concept_domain'] = 'meas'
drug_df['concept_domain'] = 'drug'
proc_df['concept_domain'] = 'proc'
cond_df['concept_domain'] = 'cond'

In [16]:
meas_df.to_csv('{}/{}_meas_df.txt'.format(output_dir, outcome_name),index=False)
drug_df.to_csv('{}/{}_drug_df.txt'.format(output_dir, outcome_name),index=False)
proc_df.to_csv('{}/{}_proc_df.txt'.format(output_dir, outcome_name),index=False)
cond_df.to_csv('{}/{}_cond_df.txt'.format(output_dir, outcome_name),index=False)